Equal-Weight S&P 500 Index Fund
Introduction & Library Imports
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund. It covers 500 largest companies. Large market capitalization -> large market base-> large weight in index

**Library Imports**
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
!pip install xlsxwriter

     |████████████████████████████████| 153kB 4.0MB/s 


In [4]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

Importing Our List of Stocks
The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course.

There's a static version of the S&P 500 constituents available here. Click this link to download them now. Move this file into the starter-files folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [5]:
stocks = pd.read_csv('/content/sp_500_stocks.csv')

In [6]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Ticker  505 non-null    object
dtypes: object(1)
memory usage: 4.1+ KB


In [8]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [12]:
from secret import IEX_CLOUD_API_TOKEN

In [13]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 94350970,
 'calculationPrice': 'close',
 'change': -1.96,
 'changePercent': -0.01482,
 'close': 126.91,
 'closeSource': 'cliofiaf',
 'closeTime': 1630911983792,
 'companyName': 'Apple Inc',
 'delayedPrice': 127.28,
 'delayedPriceTime': 1675401337282,
 'extendedChange': -0.13,
 'extendedChangePercent': -0.001,
 'extendedPrice': 127.3,
 'extendedPriceTime': 1682635925264,
 'high': 134,
 'highSource': 'rn ed 1t5de macleeipyui',
 'highTime': 1638994437513,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 128.7,
 'iexCloseTime': 1688647917022,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 126.12,
 'iexOpenTime': 1660169080454,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1665434867317,
 'latestPrice': 126.41,
 'latestSource': 'Close',
 'latestTime': 'May 21, 2021',
 'latestUpdate': 1675617494335,
 'latestVolume': 81869619,
 'low': 127

In [26]:
data['close'], data['latestPrice'], data['marketCap']

(176.29, 176.68, 84408092206)

In [28]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,176.68,84408092206,N/A


In [17]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [18]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,135.80,41382615153,N/A
1,AAL,22.94,15021322582,N/A
2,AAP,201.16,13343816728,N/A
3,AAPL,129.15,2151670195705,N/A
4,ABBV,117.17,209068675816,N/A
...,...,...,...,...
500,YUM,121.48,37273419402,N/A
501,ZBH,171.21,36193697993,N/A
502,ZBRA,501.84,27378360465,N/A
503,ZION,59.00,9633557192,N/A


**Using Batch API Calls to Improve Performance
Batch API calls are one of the easiest ways to improve the performance of your code.**

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [30]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [31]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], index = my_columns), ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,133.20,41202840179,N/A
1,AAL,22.77,15095995261,N/A
2,AAP,202.86,13397697100,N/A
3,AAPL,129.91,2185457482269,N/A
4,ABBV,120.31,207595876348,N/A
...,...,...,...,...
500,YUM,123.37,35994689343,N/A
501,ZBH,173.38,35388799587,N/A
502,ZBRA,498.56,26507589994,N/A
503,ZION,60.90,9893305362,N/A


**Calculating the Number of Shares to Buy**

In [37]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:5000


In [40]:
position_size = float(portfolio_size) / len(final_dataframe.index)
print("position_size: ", position_size)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

position_size:  9.900990099009901


,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,133.20,41202840179,0
1,AAL,22.77,15095995261,0
2,AAP,202.86,13397697100,0
3,AAPL,129.91,2185457482269,0
4,ABBV,120.31,207595876348,0
...,...,...,...,...
500,YUM,123.37,35994689343,0
501,ZBH,173.38,35388799587,0
502,ZBRA,498.56,26507589994,0
503,ZION,60.90,9893305362,0


In [41]:
final_dataframe[final_dataframe['Number Of Shares to Buy'] > 0] 

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
19,AIV,7.10,1057435109,1
111,COTY,8.69,6699659263,1
194,FTI,8.21,3879013931,1
327,NBL,8.53,4285197320,1


# Formatting Our Excel Output
We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

**Initializing our XlsxWriter Object**

In [42]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [43]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [44]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [45]:

writer.save()